<a href="https://colab.research.google.com/github/qus0in/TIL/blob/main/230928_%ED%8B%B0%EC%8A%A4%ED%86%A0%EB%A6%AC_%EC%9E%90%EB%8F%99%ED%99%94_public.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# %env access_token=...
# %env category=...
# %env visibility=0
# !echo $access_token $category $visibility

import requests
import os

class Tistory:
    '''
    티스토리 블로그 작성 자동화
    https://tistory.github.io/document-tistory-apis/
    '''
    def __init__(self, access_token):
        self.access_token = access_token
        self.get_blog_name()
        self.get_category_list()

    @property
    def default_params(self):
        return {
            'access_token': self.access_token,
            'output': 'json',
        }

    def get_blog_name(self):
        '''
        블로그 이름 받기
        https://tistory.github.io/document-tistory-apis/apis/v1/blog/list.html
        '''
        url = 'https://www.tistory.com/apis/blog/info'
        response = requests.get(url, self.default_params)
        data = response.json().get('tistory').get('item').get('blogs')[0]
        self.blog_name = data.get('name')
        print(f'blogName : {self.blog_name}')

    def get_category_list(self):
        '''
        카테고리 목록 받기
        https://tistory.github.io/document-tistory-apis/apis/v1/category/list.html
        '''
        url = 'https://www.tistory.com/apis/category/list'
        params = dict(
            blogName=self.blog_name,
            **self.default_params
        )
        response = requests.get(url, params)
        data = response.json().get('tistory').get('item').get('categories')
        self.categories = [dict(id=d['id'], name=d['name']) for d in data]
        print(f'categories : {self.categories}')

    def write_post(self, title, content, *tag):
        '''
        글 작성 : `content`의 기본 설정이 `html`임을 유의
        https://tistory.github.io/document-tistory-apis/apis/v1/post/write.html
        '''
        url = 'https://www.tistory.com/apis/post/write'
        params = dict(
            blogName=self.blog_name,
            category=os.getenv('category'),
            title=title,
            content=content,
            visibility=os.getenv('visibility'),
            tag=','.join(tag[::-1]),
            **self.default_params
        )
        response = requests.post(url, params)
        result = response.json().get('tistory').get('url')
        print(result)

t = Tistory(os.getenv('access_token'))
t.write_post(
    title='그대 곁의 그 사람 모르고 살겠죠',
    content='''
    <h1>누군가를 이렇게</h1>
    <p>아프게 하는지</p>
    '''
)

blogName : devconomy
categories : [{'id': '1113734', 'name': 'SNIPPET'}, {'id': '1144906', 'name': 'ETF'}]
https://devconomy.tistory.com/25


```python
# 테이블
df.to_html(classes='table')
```

```python
# 차트
fig.to_html().encode('utf-8')
```

```html
<iframe scrolling="no" style="border:none;" seamless="seamless"
    src="..." height="525" width="100%">
</iframe>
```

In [ ]:
%%writefile sample.csv
col1, col2, col3
1, 2, 3
4, 5, 6
7, 8, 9

Writing sample.csv


In [ ]:
'''
%env r2_account_id = ...
%env r2_access_key_id = ...
%env r2_secret_access_key = ...
%env r2_bucket = ...
%env r2_domain = ...
!echo $r2_account_id $r2_access_key_id $r2_secret_access_key $r2_bucket $r2_domain

%%writefile sample.csv
col1, col2, col3
1, 2, 3
4, 5, 6
7, 8, 9
'''

import io
import os
import hashlib
import hmac
import datetime
import requests
from requests.auth import AuthBase

class AWSV4Auth(AuthBase):
    @classmethod
    def sign(cls, key, message):
        return hmac.new(key, message.encode('utf-8'), hashlib.sha256).digest()

    @classmethod
    def get_signature_key(cls, secret_key, date_stamp, region, service):
        k_date = cls.sign(('AWS4' + secret_key).encode('utf-8'), date_stamp)
        k_region = cls.sign(k_date, region)
        k_service = cls.sign(k_region, service)
        k_signing = cls.sign(k_service, 'aws4_request')
        return k_signing

    def __init__(self, access_id, secret_key, region, service):
        self.access_id = access_id
        self.secret_key = secret_key
        self.region = region
        self.service = service

    def __call__(self, r):
        host = r.url.split('://')[1].split('/')[0]
        payload_hash = hashlib.sha256(r.body or b'').hexdigest()
        date_time = datetime.datetime.utcnow()
        amz_date = date_time.strftime('%Y%m%dT%H%M%SZ')
        date_stamp = date_time.strftime('%Y%m%d')

        canonical_request = f"{r.method}\n{r.path_url}\n\nhost:{host}\nx-amz-content-sha256:{payload_hash}\nx-amz-date:{amz_date}\n\nhost;x-amz-content-sha256;x-amz-date\n{payload_hash}"
        algorithm = 'AWS4-HMAC-SHA256'
        credential_scope = f"{date_stamp}/{self.region}/{self.service}/aws4_request"

        string_to_sign = f"{algorithm}\n{amz_date}\n{credential_scope}\n{hashlib.sha256(canonical_request.encode('utf-8')).hexdigest()}"
        signing_key = self.get_signature_key(self.secret_key, date_stamp, self.region, self.service)
        signature = hmac.new(signing_key, (string_to_sign).encode('utf-8'), hashlib.sha256).hexdigest()

        authorization_header = f"{algorithm} Credential={self.access_id}/{credential_scope}, SignedHeaders=host;x-amz-content-sha256;x-amz-date, Signature={signature}"

        r.headers['x-amz-content-sha256'] = payload_hash
        r.headers['x-amz-date'] = amz_date
        r.headers['Authorization'] = authorization_header

        return r

class R2:
    @classmethod
    def put_file(cls, filename, obj):
        aws_auth = AWSV4Auth(
            access_id=os.getenv('r2_access_key_id'),
            secret_key=os.getenv('r2_secret_access_key'),
            region='auto', service='s3')
        URL = 'https://{}.r2.cloudflarestorage.com/{}/{}'.format(
            os.getenv('r2_account_id'), os.getenv('r2_bucket'), filename,
        )
        response = requests.put(URL, data=obj, auth=aws_auth)
        return f'{os.getenv("r2_domain")}/{filename}'

filename = 'sample.csv'
with open(filename, 'rb') as f:
    print(R2.put_file(filename, f.read()))